The codes given below may be helpful for learning how to apply ARIMA function on a data-table and for making predictions based on the same. Here, the numbers of new patients added every day(till may-02,2020) in India has been used for running these codes. The codes for visualisation have also been written for getting a visual sense of data and the predictions. Three different models have been created in this file and you may create even more number of the same. Finally, the selection of the best model has been left for the readers. 

The data has been collected from this website:
https://www.worldometers.info/coronavirus/country/india/

In [ ]:
##########################################################################################

In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Read data from the csv file
data = pd.read_excel('covid_data.xlsx')

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
# creating datetime stamps to make predictions with date index
data['date'] = pd.to_datetime(data['date'], format = '%Y-%m-%d')

In [ ]:
data = pd.DataFrame(data)

In [ ]:
data = data.set_index('date')

In [ ]:
data.sort_index()

In [ ]:
data.head(2)

In [ ]:
# data visualisation in a line plot 
data.plot(figsize = (12, 4))
plt.legend(loc = 'best')
plt.title('New Patient Data')
plt.show(block = False)

In [ ]:
# data visualisation in a boxplot to check if there are outliers
fig = plt.subplots(figsize = (12,2))
ax = sns.boxplot(x = data['new_patients'], whis = 1.5)

In [ ]:
# data visualisation in a histogram
fig = data.new_patients.hist(figsize = (12,4))

In [ ]:
# splitting data in trining and test data set and leaving last empty rows for only forecast

In [ ]:
train_len = 40
test_len = 49
train = data[0:train_len]
test = data[train_len:test_len]
test_all = data[train_len:]
data_avail = data[:test_len]

In [ ]:
# importing libraries for adfuller test to check stationarity
from statsmodels.tsa.stattools import adfuller

In [ ]:
adf_test = adfuller(data_avail['new_patients'])
print('ADF Statistics: %f' % adf_test[0])
print('Critical Value @ 0.05: %.2f' % adf_test[4]['5%'])
print('p-value: %f' % adf_test[1])

In [ ]:
# importing libraries for kpss test to check stationarity
from statsmodels.tsa.stattools import kpss

In [ ]:
kpss_test = kpss(data['new_patients'])
#print('KPSS Statistics: %f' % kpss_test[0])
print('Critical Value @ 0.05: %.2f' % kpss_test[3]['5%'])
#print('p-value: %f' % kpss_test[1])

In [ ]:
### so data is not stationary

In [ ]:
# boxcox transformation and differencing to make the data set stationary
from scipy.stats import boxcox

In [ ]:
data_boxcox = pd.Series(boxcox(data['new_patients'], lmbda = 0),
                        index = data.index)

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(data_boxcox, label = 'After box cox transformation')
plt.legend(loc='best')
plt.title('After box cox transformation')
plt.show()

In [ ]:
# Differencing

In [ ]:
data_boxcox_diff = pd.Series(data_boxcox - data_boxcox.shift(),
                             index = data.index)
data_boxcox_diff.dropna(inplace = True)

In [ ]:
train_data_boxcox = data_boxcox[:train_len]
test_data_boxcox = data_boxcox[train_len:test_len]
train_data_boxcox_diff = data_boxcox_diff[:train_len-1]
test_data_boxcox_diff = data_boxcox_diff[train_len-1:test_len]

In [ ]:
# visualisation in line plot after boxcox transformation  and differencing
plt.figure(figsize=(12,4))
plt.plot(data_boxcox_diff, label = 'After box cox transformation and differencing')
plt.legend(loc='best')
plt.title('After box cox transformation and differencing')
plt.show()

In [ ]:
# adf and kpss tests after transformation
adf_test = adfuller(data_boxcox_diff)
print('ADF Statistics: %f' % adf_test[0])
print('Critical Value @ 0.05: %.2f' % adf_test[4]['5%'])
print('p-value: %f' % adf_test[1])

In [ ]:
kpss_test = kpss(data['new_patients'])
#print('KPSS Statistics: %f' % kpss_test[0])
print('Critical Value @ 0.05: %.2f' % kpss_test[3]['5%'])
#print('p-value: %f' % kpss_test[1])

In [ ]:
# ACF and PACF plots for finding p and q for ARIMA model

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

In [ ]:
plt.figure(figsize=(12,4))
plot_acf(data_boxcox_diff, ax = plt.gca(), lags = 30)
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

In [ ]:
plt.figure(figsize=(12,4))
plot_pacf(data_boxcox_diff, ax = plt.gca(), lags = 30)
plt.show()

In [ ]:
#############

In [ ]:
# AR models

In [ ]:
# importing ARIMA libraries
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
model = ARIMA(train_data_boxcox_diff, order = (10,0,0))
model_fit = model.fit()
print(model_fit.params)

In [ ]:
#  predictions based on model

In [ ]:
y_hat_ar = data_boxcox_diff.copy()
y_hat_ar['ar_forecast_boxcox_diff'] = model_fit.predict(data_boxcox_diff.index.min(), 
                                                       data_boxcox_diff.index.max())
y_hat_ar['ar_forecast_boxcox'] = y_hat_ar['ar_forecast_boxcox_diff'].cumsum()
y_hat_ar['ar_forecast_boxcox'] = y_hat_ar['ar_forecast_boxcox'].add(data_boxcox[0])
y_hat_ar['ar_forecast'] = np.exp(y_hat_ar['ar_forecast_boxcox'])

In [ ]:
# visualisation of training data, test data, and predictions
plt.figure(figsize=(12,4))
plt.plot(train['new_patients'], label = 'Train')
plt.plot(test['new_patients'], label = 'Test')
plt.plot(y_hat_ar['ar_forecast'][test.index.min():], label = 'ar_forecast')
plt.legend(loc='best')
plt.title('AR Method')
plt.show()

In [ ]:
# checking mean squared error and MAPE for evaluation of the model
from sklearn.metrics import mean_squared_error

In [ ]:
rmse = np.sqrt(mean_squared_error(test['new_patients'], 
                        y_hat_ar['ar_forecast'][test.index.min():])).round(2)
mape = np.round(np.mean(np.abs(test['new_patients'] - 
        y_hat_ar['ar_forecast'][test.index.min():])/ test['new_patients'])*100,2)
results = pd.DataFrame({'Method' : ['Auto Regression'], 'MAPE' : [mape],
                           'RMSE' : [rmse]})
results1 = results[['Method', 'MAPE', 'RMSE']]
results1

In [ ]:
# creating holt winters additive model
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
y_hat_hwa= test_all.copy()

In [ ]:
model = ExponentialSmoothing(np.asarray(train['new_patients']),
                                seasonal_periods = 7, trend = 'add', seasonal = 'add')
model_fit_hwa = model.fit(optimized = True)
print(model_fit_hwa.params)

In [ ]:
# making predictions
y_hat_hwa['hwa_forecast'] = model_fit_hwa.forecast(17)

In [ ]:
# visualisation of training data, test data, and predictions
plt.figure(figsize=(12,4))
plt.plot(train['new_patients'], label = 'Train')
plt.plot(test['new_patients'], label = 'Test')
plt.plot(y_hat_hwa['hwa_forecast'][test.index.min():test.index.max()], label = 'Holt Winters Additive Forecast')
plt.legend(loc='best')
plt.title('Holt Winters Additive Method')
plt.show()

In [ ]:
# checking mean squared error and MAPE for evaluation of the model
rmse = np.sqrt(mean_squared_error(test['new_patients'], 
                                  y_hat_hwa['hwa_forecast'][test.index.min():test.index.max()])).round(2)
mape = np.round(np.mean(np.abs(test_all['new_patients'] - 
                            y_hat_hwa['hwa_forecast'])/ 
                            test_all['new_patients'])*100,2)
results2 = pd.DataFrame({'Method' : ['Holt Winters Additive Method'],
                             'MAPE' : [mape],'RMSE' : [rmse]})
results2

In [ ]:
# creating ARIMA model with seasonaltity
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
model = SARIMAX(train_data_boxcox, order = (1,1,1), seasonal_order = (1,1,1,5))
model_fit = model.fit()
print(model_fit.params)

In [ ]:
# making predictions
y_hat_sarimax = data_boxcox_diff.copy()
y_hat_sarimax['sarimax_forecast_boxcox'] = model_fit.predict(data.index.min(),data.index.max())
y_hat_sarimax['sarimax_forecast'] = np.exp(y_hat_sarimax['sarimax_forecast_boxcox'])

In [ ]:
# visualisation of training data, test data, and predictions
plt.figure(figsize=(12,4))
plt.plot(train['new_patients'], label = 'Train')
plt.plot(test['new_patients'], label = 'Test')
plt.plot(y_hat_sarimax['sarimax_forecast'][test.index.min():test.index.max()], label = 'sarimax_forecast')
plt.legend(loc='best')
plt.title('SARIMAX Method')
plt.show()

In [ ]:
# checking mean squared error and MAPE for evaluation of the model
rmse = np.sqrt(mean_squared_error(test['new_patients'], 
                        y_hat_sarimax['sarimax_forecast'][test.index.min():test.index.max()])).round(2)
mape = np.round(np.mean(np.abs(test['new_patients'] - 
        y_hat_sarimax['sarimax_forecast'][test.index.min():])/ test['new_patients'])*100,2)
results = pd.DataFrame({'Method' : ['Seasonal Auto Regressive Integrated Moving Average'], 'MAPE' : [mape],
                           'RMSE' : [rmse]})
results3 = results[['Method', 'MAPE', 'RMSE']]
results3

In [ ]:
# comparison of all the 3 models 
results = pd.concat([results1, results2, results3])
results

In [ ]:
# making one data table with original data and predictions
data_project = pd.concat([data,y_hat_ar['ar_forecast'],y_hat_hwa['hwa_forecast'],y_hat_sarimax['sarimax_forecast']], axis = 1)
data_project.columns = ['new_patients','ar_forecast','hwa_forecast','sarimax_forecast']
data_project

In [ ]:
# to continue with the projections further, the data table may be updated with number of patients, and 
# the test may be increased simultaneously